In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS 
WITH 
asrt_list AS (
    SELECT DISTINCT 
        a.store_id,
        a.product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
    WHERE c.business_unit IN ('1500', '2000', '6000')
        AND c.store_status_desc = "Đang hoạt động"
        AND UPPER(c.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(c.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(c.store_name) NOT LIKE '%HUB%' 
        AND UPPER(c.store_name) NOT LIKE '%ONLINE%'
        AND UPPER(c.store_name) NOT LIKE '%KHO %'
        AND UPPER(c.store_name) NOT LIKE '%B2B%'
        AND a.store_id NOT LIKE 'GH%'
),
region_dc AS (
    SELECT 
        region, city,
        CASE
            WHEN UPPER(region) RLIKE r'WMT' THEN '1500'
            WHEN UPPER(region) RLIKE r'WMT' THEN '2000'
        END business_unit  
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_cfg_region_dc_city
    WHERE UPPER(region) RLIKE r'WMT'
),
main AS (
    SELECT a.region, b.city, d.store_id, a.product_id, a.vendor_id, a.vendor_subrange, a.uom, a.sell_price
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku a
    INNER JOIN region_dc b
        ON a.region = b.region
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c
        ON a.product_id = c.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store d
        ON b.city = d.city AND b.business_unit = d.business_unit
    INNER JOIN asrt_list e
        ON d.store_id = e.store_id AND a.product_id = e.product_id
    WHERE d.store_status_desc = "Đang hoạt động"
        AND UPPER(d.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(d.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(d.store_name) NOT LIKE '%HUB%' 
        AND UPPER(d.store_name) NOT LIKE '%ONLINE%'
        AND UPPER(d.store_name) NOT LIKE '%KHO %'
        AND UPPER(d.store_name) NOT LIKE '%B2B%'
        AND d.store_id NOT LIKE 'GH%'
)
SELECT *
FROM main""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh_his(
    calday DATE,
    region STRING,
    city STRING,
    store_id STRING,
    product_id STRING,
    vendor_id STRING, 
    vendor_subrange STRING , 
    uom STRING, 
    sell_price DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh_his 
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    region,
    city,
    store_id,
    product_id,
    vendor_id, 
    vendor_subrange, 
    uom, 
    sell_price
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_fresh;
""")